this notebook have a complete and very detailed example: slim_walkthrough

this is copied from tensorflow/models/research/slim. but this repo is not stable, so I copy to here for back


The most important thing is that: `slim.dataset_data_provider.DatasetDataProvider` is similiar to queue runner (in fact it use QueueRunner, but wrap it up), so in order to make thing running. we need start the queue runner. if you don't use `lim.learning.train` to train the model. you will need start queue runner yourself. with methods like: `tf.train.MonitoredSession` or `tf.train.start_queue_runners`,

slim also provide a convinient way of start queue runners

In [ ]:
# context manager 

with tf.Graph().as_default(): 
    dataset = flowers.get_split('train', flowers_data_dir)
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, common_queue_capacity=32, common_queue_min=1)
    image, label = data_provider.get(['image', 'label'])
    
    with tf.Session() as sess:    
        with slim.queues.QueueRunners(sess):
            pass


here are implmentation of slim.queues.QueueRunners in tf1.4.
Basically, all it does is use Coordinator to start the queue runner

In [9]:
from contextlib import contextmanager

@contextmanager
def QueueRunners(session):
  """Creates a context manager that handles starting and stopping queue runners.

  Args:
    session: the currently running session.

  Yields:
    a context in which queues are run.

  Raises:
    NestedQueueRunnerError: if a QueueRunners context is nested within another.
  """
  if not _queue_runner_lock.acquire(False):
    raise NestedQueueRunnerError('QueueRunners cannot be nested')

  coord = coordinator.Coordinator()
  threads = []
  for qr in ops.get_collection(ops.GraphKeys.QUEUE_RUNNERS):
    threads.extend(
        qr.create_threads(
            session, coord=coord, daemon=True, start=True))
  try:
    yield
  finally:
    coord.request_stop()
    coord.join(threads, stop_grace_period_secs=120)

    _queue_runner_lock.release()